In [ ]:
import os
import math

import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import AxesZero
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
import matplotlib.ticker as mtick
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
from scipy.stats import poisson, norm, lognorm
from scipy import optimize as opti
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.signal import savgol_filter
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [2]:
Mu = 4
Tau = 20
Sigma = 5
file = '4.0-20-5'

In [3]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]

In [4]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=gmu, scale=gsigma)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [5]:
Thres = wff.Thres
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
p = spe_pre[0]['parameters']
window = wff.window
t_auto = np.arange(window).reshape(window, 1) - np.arange(window).reshape(1, window)
mnecpu = wff.spe((t_auto + np.abs(t_auto)) / 2, p[0], p[1], p[2])

In [6]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.legend(title=r'$(\tau_l, \sigma_l)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
# ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()
ax.get_position()

Bbox([[0.125, 0.12], [0.9, 0.89]])

In [7]:
fig = plt.figure(figsize=(8, 4))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.05, right=0.97, top=0.97, bottom=0.1, wspace=0.3, hspace=0.3)
ax = fig.add_subplot(gs[0, 0])

ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.plot(1, 0, '>k', transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot(0, 1, '^k', transform=ax.get_xaxis_transform(), clip_on=False)

t = np.linspace(0, 6, 201)
ax.plot(t, lognorm.pdf(t, loc=0, s=0.3), color='darkorange')
ax.plot(t, lognorm.pdf(t, loc=3, s=0.3), color='darkblue')
ax.fill_between(t, 0, lognorm.pdf(t, loc=0, s=0.3), color='darkorange', alpha=0.5)
ax.fill_between(t, 0, lognorm.pdf(t, loc=3, s=0.3), color='darkblue', alpha=0.5)
ax.set_xlim(0, 6)
ax.set_ylim(bottom=1e-3)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r'$\mathrm{Time}$')
ax.set_ylabel(r'$\mathrm{Voltage}$')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.annotate(s='', xy=(1.5, 1), xytext=(3.5, 1), arrowprops=dict(arrowstyle='<->'))
ax.text(x=2.2, y=1.1, s=r'$\sim D_w$')
ax.text(x=0.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
ax.text(x=3.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,2,0,198.30,99.11
1,2,0,212.60,153.07
2,2,0,222.17,163.18
3,2,0,227.94,84.29
4,2,0,246.79,80.61


In [9]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 200)

In [10]:
prior = True
space = True
b_t0 = [0., 600.]
mu_t = abs(wave.sum() / gmu)
n = 2
tlist_pan = np.sort(np.unique(np.hstack(np.arange(0, window)[:, None] + np.linspace(0, 1, n, endpoint=False) - (n // 2) / n)))
A, wave_r, tlist, t0_t, t0_delta, cha, left_wave, right_wave = wff.initial_params(wave[::wff.nshannon], spe_pre[ent[i]['ChannelID']], Tau, Sigma, gmu, Thres['lucyddm'], p, is_t0=True, is_delta=False, n=n, nshannon=1)
mu_t = abs(wave_r.sum() / gmu)
factor = np.sqrt(np.diag(np.matmul(A.T, A)))
A = A / factor
p1 = mu_t * wff.convolve_exp_norm(tlist - t0_t, Tau, Sigma) / n + 1e-8
# p1 = cha / cha.sum() * mu_t + 1e-8
p1 = p1 / p1.sum() * mu_t

def likelihood(mu, t0, As_k, nu_star_k):
    a = wff.convolve_exp_norm(tlist_pan - t0, Tau, Sigma) / n + 1e-8 # use tlist_pan not tlist
    # a *= mu / a.sum()
    a *= mu
    li = -special.logsumexp(np.log(poisson.pmf(As_k, mu=a)).sum(axis=1) + nu_star_k)
    return li

def optit0mu(mu, t0, nu_star, As, mu_init=None):
    l = len(t0)
    mulist = np.arange(max(1e-8, mu - 2 * np.sqrt(mu)), mu + 2 * np.sqrt(mu), 1e-1)
    b_mu = [max(1e-8, mu - 5 * np.sqrt(mu)), mu + 5 * np.sqrt(mu)]
    # psy_star = [np.exp(nu_star[k] - nu_star[k].max()) / np.sum(np.exp(nu_star[k] - nu_star[k].max())) for k in range(l)]
    t0list = [np.arange(t0[k] - 3 * Sigma, t0[k] + 3 * Sigma + 1e-6, 0.2) for k in range(l)]
    sigmamu = None
    logLv_mu = None
    if mu_init is None:
        mu_init = np.empty(l)
        for k in range(l):
            mu_init[k] = mulist[np.array([likelihood(mulist[j], t0[k], As[k], nu_star[k]) for j in range(len(mulist))]).argmin()]
            t0_init = t0list[k][np.array([likelihood(mu_init[k], t0list[k][j], As[k], nu_star[k]) for j in range(len(t0list[k]))]).argmin()]
            likelihood_x = lambda x, As, nu_star: likelihood(x[0], x[1], As, nu_star)
            t0[k] = opti.fmin_l_bfgs_b(likelihood_x, args=(As[k], nu_star[k]), x0=[mu_init[k], t0_init], approx_grad=True, bounds=[b_mu, b_t0], maxfun=50000)[0][1]
        Likelihood = lambda mu: np.sum([likelihood(mu, t0[k], As[k], nu_star[k]) for k in range(l)])
        mu, fval, _ = opti.fmin_l_bfgs_b(Likelihood, x0=[np.mean(mu_init)], approx_grad=True, bounds=[b_mu], maxfun=50000)
    return mu, t0, [sigmamu, mulist, logLv_mu, fval]

xmmse_star, nu_star, T_star, c_star, d_max_i, num_i = wff.fbmpr_fxn_reduced(wave_r, A, spe_pre[cid]['std'] ** 2, (gsigma * factor / gmu) ** 2, factor, len(p1), p1=p1, truth=truth, i=i, left=left_wave, right=right_wave, tlist=tlist, gmu=gmu, para=p, prior=prior, space=space)
As = np.zeros((num_i, len(tlist_pan)))
As[:, np.isin(tlist_pan, tlist)] = c_star
assert sum(np.sum(As, axis=0) > 0) > 0
maxindex = nu_star.argmax()
spacefactor = 0
priorfactor = 0
if not space:
    spacefactor = np.array([-0.5 * np.log(np.linalg.det(wff.Phi(y, A, c_star[j], mus, sig2s, sig2w, p1))) for j in range(num_i)])
if prior:
    priorfactor = -poisson.logpmf(c_star, p1).sum(axis=1)
nu_star = nu_star + priorfactor + spacefactor
mu, t0, _ = optit0mu(mu_t, [t0_t], [nu_star], [As])
mu_i, t0_i, _ = optit0mu(mu_t, [t0_t], [np.array([0.])], [As[maxindex][None, :]])
xmmse_most = np.clip(xmmse_star[maxindex], 0, np.inf)
pet = np.repeat(tlist[xmmse_most > 0], c_star[maxindex][xmmse_most > 0])
cha = np.repeat(xmmse_most[xmmse_most > 0] / factor[xmmse_most > 0] / c_star[maxindex][xmmse_most > 0], c_star[maxindex][xmmse_most > 0])
pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
wave_ylim = ax.get_ylim()
fig.savefig('Note/figures/fbmp.pgf')
fig.savefig('Note/figures/fbmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [199.  212.5 222.  228.  246.5], Weight = [0.6251461  0.88673444 1.03530568 0.53611226 0.52423241]
wdist = 0.5948299604432856, cdiff = -3.065970479297077
RSS = 15.778199437200463
-3.438693135568286


In [11]:
# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.plot(wave, label='Waveform')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
# ax.set_xlim(0, len(wave))
# wave_ylim = ax.get_ylim()
# ax.set_ylim(wave_ylim[0] * 1.05, wave_ylim[1] * 1.05)
# ax.legend()
# fig.savefig('Note/figures/wave.pgf')
# fig.savefig('Note/figures/wave.pdf')
# fig.savefig('Note/figures/wave.png')
# fig.clf()
# plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge', linewidth=1.0)
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax2.set_ylim(wave_ylim[0] / 30, wave_ylim[1] / 30)
ax.plot(wave, label='Waveform')
# ax.set_xlim(xmin, xmax)
ax.set_xlim(0, len(wave) / 2)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
align.yaxes(ax, 0, ax2, 0)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.savefig('Note/figures/wave.png')
fig.clf()
plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
fig.savefig('Note/figures/spe.png')
plt.close()

# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Charge}$')
# ax.set_xlim(0, len(wave))
# ax.set_ylim(wave_ylim[0] / 20, wave_ylim[1] / 20)
# ax.axhline(y=0, color='k', linestyle='dashed', alpha=0.5)
# ax.legend()
# fig.savefig('Note/figures/charge.pgf')
# fig.savefig('Note/figures/charge.pdf')
# fig.savefig('Note/figures/charge.png')
# fig.clf()
# plt.close(fig)

In [12]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5 / gmu)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [13]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise

600.612084468333
580.2596844449434


In [14]:
t = np.load('result/takara/char/Channel00/cnn_testing_record_2021-07-30_17:15:10.npz')['arr_0']
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax.plot(np.arange(1, len(t)+1), t, label=r'$D_w$', color='C1')
ax.set_xlabel(r'$\mathrm{epoch}$')
ax.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [15]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [195 196 197 198 199 200 201 202 203 205 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 244 245 246 247 248 249 250 251], Weight = [0.03584503 0.04490778 0.05110216 0.05317444 0.05945964 0.06187086
 0.05573373 0.04979399 0.04034561 0.03863693 0.04677279 0.04984948
 0.08103905 0.09004558 0.09130871 0.10570219 0.07561306 0.0813948
 0.08146041 0.06363937 0.09169068 0.12561056 0.12280308 0.12547056
 0.12715693 0.12155163 0.13276326 0.13767504 0.12497751 0.12037777
 0.11556223 0.10702699 0.09377507 0.08716678 0.06529191 0.04768089
 0.04531403 0.05371381 0.03920505 0.0472688  0.05136636 0.04668365
 0.0619234  0.05624993 0.04690911 0.04493587 0.04083298]
wdist = 3.1087242325850317, cdiff = -14.081985436358435
R

In [16]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for j in range(len(loc)):
    ax.annotate('', xy=(loc[j], amp[j]+5), xytext=(loc[j], amp[j]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [200 213 223 248], Weight = [0.61116178 1.04412873 1.20068987 0.55563809]
wdist = 2.3999066086436542, cdiff = -34.41134854495826
RSS = 266.9323131608509
-1.9290449992176946


In [17]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [198 199 200 210 211 212 213 214 220 221 222 223 224 225 226 227 246 247
 248], Weight = [0.14995905 0.16136076 0.14971286 0.15427833 0.19532405 0.20962768
 0.19607729 0.16101529 0.1266714  0.1781401  0.22298341 0.24987798
 0.25239387 0.23044018 0.18988035 0.14052016 0.14142437 0.14293999
 0.12627225]
wdist = 2.029237712534919, cdiff = -39.64630385299262
RSS = 124.67317506427365
-1.1596268328024166


In [18]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [199 212 213 222 228 247], Weight = [0.45509249 0.48911629 0.54882022 1.04444101 0.43353076 0.50499002]
wdist = 1.0994500816814046, cdiff = -24.111976439534768
RSS = 70.25593260038926
-0.7472914331635252


In [19]:
with h5py.File('result/takara/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/takara.pgf')
fig.savefig('Note/figures/takara.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [198 199 212 213 222 228 247], Weight = [0.57807553 0.07023228 0.37157281 0.51505843 1.00229091 0.57173777
 0.51790512]
wdist = 0.6408859192063601, cdiff = 0.028682484579321738
RSS = 10.958349058233402
-3.449534874550608


In [20]:
with h5py.File('result/mcmc/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/mcmc.pgf')
fig.savefig('Note/figures/mcmc.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [195.  196.  196.5 197.  198.  198.5 199.  199.5 200.  201.5 202.  202.5
 203.  209.5 210.  212.5 213.  214.  217.5 219.  221.5 222.  224.5 227.5
 228.  229.5 230.  230.5 246.  246.5 247.5 251. ], Weight = [0.00006055 0.0000123  0.00002088 0.00001721 0.00001772 0.62653152
 0.00004476 0.00002066 0.00000857 0.00000206 0.00002402 0.00000339
 0.00001742 0.00003347 0.00000176 0.88393095 0.0000119  0.00002502
 0.00000106 0.00000154 0.49642209 0.49695779 0.00000266 0.5833009
 0.00001748 0.00001247 0.00001806 0.00000023 0.00003033 0.52634848
 0.00002249 0.00000373]
wdist = 0.695623374533694, cdiff = -2.043179450830088
RSS = 8.949425328589244
-2.9015390278386803


In [21]:
# pet, pwe = wff.xiaopeip(wave, spe_pre[cid], eta=0)
pet, pwe = wff.xiaopeip(wave, spe_pre[cid], Tau, Sigma, Thres['lucyddm'], p, eta=0)
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [198.30013112 212.60177919 222.16899484 227.93643283 246.79015495], Weight = [0.61947338 0.95673715 1.01986389 0.52680905 0.50381012]
truth RSS = 1003.2304605600609
HitPosInWindow = [195. 198. 199. 202. 212. 213. 221. 222. 227. 228. 246. 247.], Weight = [0.07723325 0.18981735 0.34506414 0.03396807 0.4016685  0.49060024
 0.05036373 0.97605184 0.03227011 0.50920533 0.09909962 0.42226446]
wdist = 0.676499343758253, cdiff = 0.14608994582556287
RSS = 7.639666451461602
-3.225718943402512


In [22]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

    L = len(dt)
    data = dt['wdist']
    fig = plt.figure(figsize=(8, 6))
    ax1 = fig.add_axes((.125, .12, .6, .77))
    boxdict = ax1.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax1.set_xticks(np.arange(1, 16, 2))
    ax1.set_xticklabels(np.arange(1, 16, 2).astype(str))
    ax1.plot(np.arange(1, l + 1), wdist_stats[:, 0], label=r'$D_w$')
    ax1.set_xlim(0, l + 1)
    ax1.set_ylim(0, max([boxdict['whiskers'][2 * i + 1].get_xydata()[1, 1] for i in range(l)]) * 1.05)
    ax1.set_xlabel(r'$N_{\mathrm{PE}}$')
    ax1.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
    ax1.legend()
    ax2 = fig.add_axes((.725, .12, .175, .77))
    ax2.hist(data, bins=np.arange(0, data.max(), np.percentile(data, 98) / 40), density=1, orientation='horizontal')
    ax2.set_xlabel(r'$\mathrm{arb.\ unit}$')
    ax2.set_xlim(0, ax2.get_xlim()[1] * 1.05)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_ylim(ax1.get_ylim())
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)

In [23]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
tlist = np.arange(-50, 200)

for mu in tqdm(Mu * wff.convolve_exp_norm(tlist, Tau, Sigma)):
    for i in range(1, 15):
        pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))

100%|██████████| 250/250 [00:01<00:00, 126.80it/s]


In [24]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fbmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
t = np.arange(0, 1000, 0.1) / gmu
# ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fbmp':'\mathrm{FBMP}', 'fbmpwave':'\mathrm{FBMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/' + file + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist.png')
plt.savefig('Note/figures/recchargehist.pdf')
plt.savefig('Note/figures/recchargehist.pgf')
plt.show()

In [25]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
b = 0.5
tlist = np.arange(-50, 200, b)

for mu in tqdm(25 * wff.convolve_exp_norm(tlist, Tau, Sigma) * b):
    for i in range(1, 15):
        pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))

100%|██████████| 500/500 [00:03<00:00, 128.21it/s]


In [26]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fbmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fbmp':'\mathrm{FBMP}', 'fbmpwave':'\mathrm{FBMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/15.0-20-5' + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist25.png')
# plt.savefig('Note/figures/recchargehist.pdf')
# plt.savefig('Note/figures/recchargehist.pgf')
plt.show()